In [1]:
from IPython.display import HTML

HTML('''<script>
code_show=true; 
function code_toggle() {
 if (code_show){
 $('div.input').hide();
 } else {
 $('div.input').show();
 }
 code_show = !code_show
} 
$( document ).ready(code_toggle);
</script>
<form action="javascript:code_toggle()"><input type="submit" value="Click here to toggle on/off the raw code."></form>''')

In [2]:
# import os
import sqlite3 as sql
from sqlite3 import Error
import numpy as np
import pandas as pd
import plotly.graph_objects as go
import plotly.offline as py
import matplotlib.pyplot as plt


Creating connection to database

In [3]:
def create_connection(db_file, delete_db=False):
    if delete_db and os.path.exists(db_file):
        os.remove(db_file)
    conn = None
    try:
        conn = sql.connect(db_file)
        conn.execute("PRAGMA foreign_keys = 1")
    except Error as e:
        print(e)

    return conn




Creating neccessary tables from the existing tables


In [4]:
## creating neccessary tables from the existing tables
db_file = 'project.db'
conn = create_connection(db_file)
with conn :
    cur = conn.cursor()
    rows = cur.execute("SELECT name FROM sqlite_master WHERE type='table';").fetchall()
    list_of_tables_in_db = []
    for row in rows:
        list_of_tables_in_db.append(row[0])
    filter_name = 'Median'
    temp_list = []
    for i in list_of_tables_in_db :
        cur.execute('PRAGMA TABLE_INFO({})'.format(i));
        sd = cur.fetchall();
        list_of_columns = [tup[1] for tup in sd];
        line = ''
        for j in list_of_columns :
            temp = j +' = CASE '+ j +' WHEN '+'\'\''+ ' THEN NULL ELSE '+j+' END,'
            line += temp
        cur.execute('UPDATE {table_name} SET {line} ;'.format(table_name = i,line = line[:-1]))
        if not i.endswith('_M') :
            new_table_name = i + '_B_M'
            cur.execute('CREATE TABLE {new} AS SELECT * FROM {old} where Uncertainty_bounds = ?;'.format(new = new_table_name, old = i),(filter_name,))
#             cur.execute('ALTER TABLE {new} DROP  COLUMN Uncertainty_bounds ;'.format(new = new_table_name))
            temp_list.append(new_table_name)
    list_of_tables_in_db.extend(temp_list)
    conn.commit()
conn.close()

OperationalError: table A_5_14_DEATHS_B_M already exists


Storing the names of all tables in a list for furthur use


In [ ]:
## stores the names of all countries
db_file = 'project.db'
conn = create_connection(db_file)

with conn :
    cur = conn.cursor()
    rows = cur.execute("SELECT name FROM sqlite_master WHERE type='table';").fetchall()
    list_of_tables_in_db = []
    for row in rows:
        list_of_tables_in_db.append(row[0])

conn.close()


Storing country names in both Case sensitive and Case insensitive for further use


In [ ]:
##storing name if the countries in a list for both case sensitive and insensitive
db_file = 'project.db'
conn = create_connection(db_file)
country_names = pd.read_sql_query('select distinct(Country_Name) from IMR_MALE_M',conn).set_index('Country_Name').transpose()
country_names_list = country_names.keys()
case_insensitive_country_names = [i.lower() for i in country_names_list]




Creating dictionary of dataframes for all years of Number of deaths in each category


In [ ]:
### deaths_data
deaths_data_every_year={}
db_file = 'project.db'
conn = create_connection(db_file)
for yr in np.arange(1950,2019):
    i = str(yr)
    with conn :
        cur = conn.cursor()
        deaths_data = pd.read_sql_query("SELECT A.ISO_Code AS ISO_Code, A.Country_Name AS Country , A.y_"+i+" AS AGE_5_14 ,I.y_"+i+" AS INFANT ,N.y_"+i+" AS NEONATAL ,U.y_"+i+" AS UNDER_5  FROM A_5_14_DEATHS_B_M A INNER JOIN INFANT_DEATHS_B_M I USING (ISO_Code) INNER JOIN NEONATAL_DEATHS_B_M N USING (ISO_Code) INNER JOIN U5MR_DEATHS_B_M U USING (ISO_Code)",conn);
        deaths_data_every_year[yr]=deaths_data
conn.close()


Creating dictionary of dataframes for all years of Mortality Rates in each category


In [ ]:
### mortality_rate_data
mortality_rate_data_every_year={}
db_file = 'project.db'
conn = create_connection(db_file)
for yr in np.arange(1950,2019):
    i = str(yr)
    with conn :
        cur = conn.cursor()
        mortality_rate_data = pd.read_sql_query("SELECT A.ISO_Code AS ISO_Code, A.Country_Name AS Country , A.y_"+i+" AS AGE_5_14 ,I.y_"+i+" AS INFANT ,N.y_"+i+" AS NEONATAL ,U.y_"+i+" AS UNDER_5  FROM A_5_14_MORTALITY_RATE_B_M A INNER JOIN INFANT_MORTALITY_RATE_B_M I USING (ISO_Code) INNER JOIN NEONATAL_MORTALITY_RATE_B_M N USING (ISO_Code) INNER JOIN U5MR_MORTALITY_RATE_B_M U USING (ISO_Code)",conn);
        mortality_rate_data_every_year[yr]=mortality_rate_data
conn.close()


Function to display pie chart of Number of deaths for a specific year and country


In [ ]:
## function to display pie chart for a specific year and country
def pie_chart_country_year(country,year = 2018):
    
    df = deaths_data_every_year[year][['Country', 'AGE_5_14', 'INFANT', 'NEONATAL', 'UNDER_5']]
    data = df.set_index('Country').transpose();
    
    country = country_names_list[case_insensitive_country_names.index(country)]
    if not data[country].isna().values.any():
        fig = go.Figure(data = [go.Pie(labels = data[country].keys(),values = list(data[country]))])
        fig.show()
#         print(plot)
    else :
        print("No data available for this country for the year required")



Function to display pie chart of Number of deaths for a specific year to compare among all countries



In [ ]:
##function to display pie chart for a specific year to compare among all countries

def pie_chart_for_an_year(year = 2018):
    
#     year_column = 'y_' + str(year)
#     required_data = "SELECT A.Country_Name AS Country , A."+year_column+" AS AGE_5_14 ,I."+year_column+" AS INFANT ,N."+year_column+" AS NEONATAL ,U."+year_column+" AS UNDER_5  FROM A_5_14_DEATHS_B_M A INNER JOIN INFANT_DEATHS_B_M I USING (ISO_Code) INNER JOIN NEONATAL_DEATHS_B_M N USING (ISO_Code) INNER JOIN U5MR_DEATHS_B_M U USING (ISO_Code)"
    df = deaths_data_every_year[year][['Country', 'AGE_5_14', 'INFANT', 'NEONATAL', 'UNDER_5']]
    df['Total Deaths'] = df ['AGE_5_14'] + df ['INFANT'] + df ['NEONATAL'] + df ['UNDER_5']
    df_total_deaths = df[['Country','Total Deaths']]
    total = 'Total Deaths'
    data = df_total_deaths.set_index('Country')
    d_c = data.transpose()
    fig = go.Figure(data = [go.Pie(labels = d_c.keys(),values = list(data[total]))])
    fig.show()



Function to display pie chart of Mortality Rates for a specific year to compare among all countries


In [ ]:
##function to display pie chart for a specific year to compare among all countries

def pie_chart_of_mortality_for_an_year(year = 2018):
    
#     year_column = 'y_' + str(year)
#     required_data = "SELECT A.Country_Name AS Country , A."+year_column+" AS AGE_5_14 ,I."+year_column+" AS INFANT ,N."+year_column+" AS NEONATAL ,U."+year_column+" AS UNDER_5  FROM A_5_14_DEATHS_B_M A INNER JOIN INFANT_DEATHS_B_M I USING (ISO_Code) INNER JOIN NEONATAL_DEATHS_B_M N USING (ISO_Code) INNER JOIN U5MR_DEATHS_B_M U USING (ISO_Code)"
    df = mortality_rate_data_every_year[year][['Country', 'AGE_5_14', 'INFANT', 'NEONATAL', 'UNDER_5']]
    df['Total Deaths'] = df ['AGE_5_14'] + df ['INFANT'] + df ['NEONATAL'] + df ['UNDER_5']
    df_total_mortality_rate = df[['Country','Total Deaths']]
    total = 'Total Deaths'
    data = df_total_mortality_rate.set_index('Country')
    d_c = data.transpose()
    fig = go.Figure(data = [go.Pie(labels = d_c.keys(),values = list(data[total]))])
    fig.show()



Defining slider


In [ ]:
## defining slider
keys=deaths_data_every_year[1960].keys()
keys=['AGE_5_14', 'INFANT', 'NEONATAL', 'UNDER_5']

slider_min=1950
slider_max=2018
slider_step=1


Defining function which takes input and returns suggestions for the user


In [ ]:
## function which takes input and returns suggestions for the user

def possible_countries(country):

    d = ''
    possible_country_list = []
    possible_country_list = case_insensitive_country_names.copy()
    for i in country.lower() :
        out = []
        d += i
#         print(d)
        for j in possible_country_list:

            if j.startswith(d):

                out.append(j)

        if len(out) == 0:

            return possible_country_list

        out.sort()
        possible_country_list = out.copy()

    return possible_country_list



In [ ]:
####     deaths (trend) over all years for all countries 
# deaths_data_every_year={}
# db_file = 'project.db'
# conn = create_connection(db_file)
# for yr in np.arange(1950,2019):
#     i = str(yr)
#     with conn :
#         cur = conn.cursor()
#         deaths_data = pd.read_sql_query("SELECT A.ISO_Code AS ISO_Code, A.Country_Name AS Country , A.y_"+i+" AS AGE_5_14 ,I.y_"+i+" AS INFANT ,N.y_"+i+" AS NEONATAL ,U.y_"+i+" AS UNDER_5  FROM A_5_14_DEATHS_B_M A INNER JOIN INFANT_DEATHS_B_M I USING (ISO_Code) INNER JOIN NEONATAL_DEATHS_B_M N USING (ISO_Code) INNER JOIN U5MR_DEATHS_B_M U USING (ISO_Code)",conn);
#         deaths_data_every_year[yr]=deaths_data
# conn.close()
#
output=[]
for step in np.arange(slider_min, slider_max, slider_step):
    yr=step
    o1=deaths_data_every_year[yr].copy()
    o1[keys]=o1[keys].apply(pd.to_numeric, errors='coerce')
    output.append(o1[keys].sum())
output=pd.DataFrame(output)


fig = go.Figure()
x=np.arange(slider_min, slider_max, slider_step)
for k in output.columns:
    fig.add_trace(go.Scatter(x=x, y=output[k],
                        mode='lines+markers',
                        name=k))
    
fig.update_layout(
    title=go.layout.Title(
        text='Child Mortality Worldwide',
        xref="paper",
        x=0
    ),paper_bgcolor='rgba(0,0,0,0)',
plot_bgcolor='rgba(0,0,0,0)',
    xaxis=go.layout.XAxis(
        title=go.layout.xaxis.Title(
            text="Year",
            font=dict(
                family="Courier New, monospace",
                size=18,
                color="#7f7f7f"
            )
        )
    ),
    yaxis=go.layout.YAxis(
        title=go.layout.yaxis.Title(
            text="# Deaths",
            font=dict(
                family="Courier New, monospace",
                size=18,
                color="#7f7f7f"
            )
        )
    )
)

fig.show()



Displays trend in mortality rates for all the countries over all years


In [ ]:
# Trend in mortality rates for all the countries over all years

fig={}
for kk in keys: #iter 1
    fig[kk] = go.Figure()

# x=np.arange(slider_min, slider_max, slider_step)
for country in range(len(mortality_rate_data_every_year[1960])):
    
    for kk in keys: #iter 2
        c_out=[ mortality_rate_data_every_year[yr][kk].loc[country] for yr in np.arange(1950,2019)]
        fig[kk].add_trace(go.Scatter(x=np.arange(1960,2019,1), y=c_out,
                            mode='lines',
                            opacity=0.5,
                            name=mortality_rate_data_every_year[yr]['Country'].loc[country] ))
    
    fig[kk].update_layout(
        title=go.layout.Title(
            text='Child Mortality Worldwide',
            xref="paper",
            x=0
        ),paper_bgcolor='rgba(0,0,0,0)',
    plot_bgcolor='rgba(0,0,0,0)',
        xaxis=go.layout.XAxis(
            title=go.layout.xaxis.Title(
                text="Year",
                font=dict(
                    family="Courier New, monospace",
                    size=18,
                    color="#7f7f7f"
                )
            )
        ),
        yaxis=go.layout.YAxis(
            title=go.layout.yaxis.Title(
                text="# Mortality Rate",
                font=dict(
                    family="Courier New, monospace",
                    size=18,
                    color="#7f7f7f"
                )
            )
        )
    )

fig[kk]


World map visualization



In [ ]:
#### world map visualization
yr = 1990
keys=mortality_rate_data_every_year[1960].keys()
keys=['AGE_5_14', 'INFANT', 'NEONATAL', 'UNDER_5']

slider_min=1950
slider_max=2018
slider_step=1


fig1={}
for kk in keys: #iter 1
    fig1[kk] = go.Figure()
for step in np.arange(slider_min, slider_max, slider_step):
    yr=step
    output=mortality_rate_data_every_year[yr].copy()
    output[keys]=output[keys].apply(pd.to_numeric, errors='coerce')
#     output[keys].loc[0]=output[keys].loc[0].fillna(0)
#     output=output.fillna(method='ffill')
#     output=output.fillna(method='bfill')
    N=output.shape[0]
    
    t=["",np.dtype('>U64')]*N
    for i in range(N):
        temp=np.dtype('>U64')
        temp='Year:'+str(yr)+"<br>"
        for kk in ['Country','AGE_5_14', 'INFANT', 'NEONATAL', 'UNDER_5']:
            temp=temp+str(kk)+":"+output[kk].astype('str')[i]+"<br>"
        t[i]=temp

    for kk in keys: #iter 2
        if kk in keys:
            sizes=50*(output[kk]-min(output[kk]))/(max(output[kk])-min(output[kk]))
        else:
            sizes=5

        fig1[kk].add_trace(go.Choropleth(
                    locations=output['ISO_Code'],
                    z = output['UNDER_5'],
                    text = output['Country'],
                    colorscale = 'Blues',
                    autocolorscale=False,
                    reversescale=True,
                    marker_line_color='darkgray',
                    marker_line_width=0.5,
                    name="Year = " + str(step),
                #     colorbar_tickprefix = '',
#                     colorbar_title = '$',
                    ))


for kk in keys: #iter 3
    fig1[kk].data[0].visible = True
    # Create and add slider
    steps = []
    for i in range(len(fig1[kk].data)):
        step = dict(
            method="restyle",
            args=["visible", [False] * len(fig1[kk].data)],
        )
        step["args"][1][i] = True  # Toggle i'th trace to "visible"
        steps.append(step)

    sliders = [dict(
        active=10,
        currentvalue={"prefix": "Year: "},
        pad={"t": 50},
        steps=steps
    )]

    fig1[kk].update_layout(
        sliders=sliders
    )
    fig1[kk].update_layout(
        title=go.layout.Title(
            text='Relative child mortality across the globe',
            xref="paper",
            x=0
        ),paper_bgcolor='rgba(0,0,0,0)',
    plot_bgcolor='rgba(0,0,0,0)',
        xaxis=go.layout.XAxis(
            title=go.layout.xaxis.Title(
                text="x Axis",
                font=dict(
                    family="Courier New, monospace",
                    size=18,
                    color="#7f7f7f"
                )
            )
        ),
        yaxis=go.layout.YAxis(
            title=go.layout.yaxis.Title(
                text="y Axis",
                font=dict(
                    family="Courier New, monospace",
                    size=18,
                    color="#7f7f7f"
                )
            )
        )
    )


In [ ]:
fig1[kk]


Displays pie chart for Number of deaths for any year with the help of slider


In [ ]:
## displays pie chart of deaths for any year ( slider is provided)

from __future__ import print_function
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets
db_file = 'project.db'
conn = create_connection(db_file)
def f(year) :
    with conn :
        pie_chart_for_an_year(year)
interact(f, year =widgets.IntSlider(min=1950, max=2018, step=1));
        


Displays pie chart for Mortality rate for any year with the help of slider



In [ ]:
## displays pie chart for mortality rate for any year ( slider is provided)

from __future__ import print_function
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets
db_file = 'project.db'
conn = create_connection(db_file)
def f(year) :
    with conn :
        pie_chart_of_mortality_for_an_year(year)
interact(f, year =widgets.IntSlider(min=1950, max=2018, step=1));
        
        



Displays pie chart of Number of deaths for a country in a particular year


In [ ]:
## run to display desired pie chart of deaths for a country in a particular year

class invalid(Exception):
    pass
class future(Exception):
    pass
class past(Exception):
    pass
# class invalid(Exception):
#     pass
db_file = 'project.db'
conn = create_connection(db_file)

def input_the_country_name():
    
    try:

        country = input('Enter the country name : ')
        country = country.lower()
        if country not in case_insensitive_country_names :
            raise invalid
        else :

            year = input('Enter the year you are looking for : ')
            try :
                year = int(year)
                if year > 2018 :
                    raise future
                elif year < 1950 :
                    raise past
                else :
                    with conn :

                        pie_chart_country_year(country,year)
            except ValueError :
                print("Please print a valid year")
            except future:
                print("future data not available")
            except past:
                print("Sorry! The power of the data has not been realized yet . Please enter any year after 1950")

    except invalid:
        possible_country_list = possible_countries(country)

        if len(possible_country_list) == len(country_names_list) :
            print("Country entered is invalid ")
        else :
            print("Entered country is wrong . Please check the below suggestions :\n",[i.title() for i in possible_country_list])
            input_the_country_name()
input_the_country_name()





Displays pie chart of Number of deaths for a country in a particular year with the help of slider to compare the trends throughout the years.


In [ ]:
## displays pie chart by taking country name as input and a slider is provided

from __future__ import print_function
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets


class invalid(Exception):
    pass


db_file = 'project.db'
conn = create_connection(db_file)
def input_the_country_name():
    try:

        country = input('Enter the country name : ')
        country = country.lower()
        if country not in case_insensitive_country_names :
            raise invalid
        else :
            def f(year) :
                with conn :
                    pie_chart_country_year(country,year)
            interact(f, year =widgets.IntSlider(min=1950, max=2018, step=1));


    except invalid:
        possible_country_list = possible_countries(country)

        if len(possible_country_list) == len(country_names_list) :
            print("Country entered is invalid ")
        else :
            print("Entered country is wrong . Please check the below suggestions :\n",[i.title() for i in possible_country_list])
            input_the_country_name()
input_the_country_name()




Creating pandas data frame to visualize a line graph


In [ ]:
# creating pandas data frame to visualize a line graph
a= []
b= []
db_file = 'project.db'
conn = create_connection(db_file)

for i in list_of_tables_in_db :
    if not i.endswith('_M'):
        with conn :
            cur = conn.cursor()
            sq = 'SELECT *  FROM ' + i +';'
            df_name = i.lower()
            df_name = pd.read_sql_query(sq,conn);
            a.append(i.lower())
            b.append(df_name)

dic = dict(zip(a,b))
            


Defined a function to plot line graph of mortality rates for a country over years


In [ ]:
# function to plot line graph to mortality rates for a country over years
def line_graph_over_the_years (country):
    c = country.title()
    years = [i for i in range(1950,2019)]
    for i in dic.keys():
        if 'mortality_rate' in i :
            df=dic[i]
            category = i.split("_m")
            lower = np.array(df[df['Uncertainty_bounds']=='Lower'][df[df['Uncertainty_bounds']=='Lower']['Country_Name']==c])[0][3:]
            upper = np.array(df[df['Uncertainty_bounds']=='Upper'][df[df['Uncertainty_bounds']=='Upper']['Country_Name']==c])[0][3:]
            median = np.array(df[df['Uncertainty_bounds']=='Median'][df[df['Uncertainty_bounds']=='Median']['Country_Name']==c])[0][3:]
            trace0 = go.Scatter(
                x = years, 
                y = upper,
                fill= None,
                mode='lines',
                name='Upper',
                line=dict(
                    color='rgb(225,0,0 )',
                )
            )
            trace1 = go.Scatter(
                x = years, 
                y = lower,
                fill='tonexty',
                mode='lines',
                name='Lower',
                line=dict(
                    color='rgb(255, 0, 0)',
                )
            )
            trace2 = go.Scatter(
                x = years, 
                y = median,
                name='Average',
                line=dict(
                    color='rgb(1, 1, 1)',
                )
            )
            data = [trace0, trace1, trace2]

            layout = go.Layout(
                xaxis=dict(title='year'),
                yaxis=dict(title=category[0].title()+' Moratality Rate'),
                title=category[0].title()+' Mortality rate for ' + c + ' over the years',
                showlegend = False)

            fig = go.Figure(data=data, layout=layout)
            py.iplot(fig)
            


Displaying line graphs for a country over years with range (Upper and Lower) for all four categories


In [ ]:
db_file = 'project.db'
conn = create_connection(db_file)
def input_the_country_name():
    try:

        country = input('Enter the country name : ')
#         org_country = country
        country = country.lower()
        if country not in case_insensitive_country_names :
            raise invalid
        else :
            line_graph_over_the_years (country)
    except invalid:
        possible_country_list = possible_countries(country)

        if len(possible_country_list) == len(country_names_list) :
            print("Country entered is invalid ")
        else :
            print("Entered country is wrong . Please check the below suggestions :\n",[i.title() for i in possible_country_list])
            input_the_country_name()
input_the_country_name()
